In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from matplotlib.backends.backend_pdf import PdfPages
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
#pd.options.mode.copy_on_write = True
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM
import multiprocessing as mp
from tqdm import tqdm
cosmo = FlatLambdaCDM(name='Planck18', H0=67.66, Om0=0.30966, Tcmb0=2.7255, Neff=3.046, m_nu=[0.  , 0.  , 0.06]* u.eV, Ob0=0.04897)
cores = 8

In [2]:
sys.path.append('../UploadToFritz')
import GetLightcurve

def get_fphot(data):
    objid, crossmatch = data
    df_photometry = GetLightcurve.full_dataframe([objid, crossmatch])[['STANDARD_NAME', 'SEMESTER_NAME', 'SCIENCE_NAME', 'TEMPLATE_NAME', 'MJD_OBS', 'FILTER', 'DETPOS',
       'X_OBJ', 'Y_OBJ', 'X_FPHOT', 'Y_FPHOT', 'X_nSTAR', 'Y_nSTAR', 'X_nGAL',
       'Y_nGAL', 'X_ALT', 'Y_ALT', 'MAG_ALT', 'MAGERR_ALT', 'SNR_ALT',
       'CNNSCORE_ALT', 'ZMIN_TEMP', 'ZMAX_TEMP', 'ZMIN_SCI', 'ZMAX_SCI',
       'ZMIN_DIFF', 'ZMAX_DIFF', 'APER_FPHOT', 'ZP_FPHOT', 'LIM_MAG3',
       'LIM_MAG5', 'MAG_FPHOT', 'MAGERR_FPHOT', 'SNR_FPHOT', 'STATUS_FPHOT',
       'REAL', 'LIMIT']]
    
    df_photometry.drop_duplicates(subset = ["MAG_FPHOT", "MAGERR_FPHOT", "MJD_OBS", "FILTER"], inplace = True)
    
    return df_photometry

In [3]:
crossmatch_df =  pd.read_csv('../MatchToDESI/candidates_desi.csv')
interesting_df =  pd.read_csv('../MatchToDESI/interesting_desi.csv')

In [4]:
queue = [[i, interesting_df] for i in interesting_df["STANDARD_NAME"]]
pool = mp.Pool(processes=cores)
interesting_lightcurve_dfs = list(tqdm(pool.imap_unordered(get_fphot, queue), total = len(queue)))
interesting_lightcurves_df = pd.concat(interesting_lightcurve_dfs)

100%|██████████| 649/649 [00:12<00:00, 51.72it/s]


In [5]:
interesting_lightcurves_table = Table.from_pandas(interesting_lightcurves_df)
interesting_table = Table.from_pandas(interesting_df)

In [6]:
np.unique(interesting_lightcurves_table["FILTER"])
filter_dict = {"g" : "green", "i" : "indigo" , "r" : "red", "z" : "orange"}

In [8]:
p = PdfPages('interesting_transients.pdf')
for row in interesting_table:
    curfig = plt.figure(figsize=[10,6])
    lc = interesting_lightcurves_table[np.where(interesting_lightcurves_table["STANDARD_NAME"] == row["STANDARD_NAME"])]
    lc_real = lc[lc["REAL"]]
    brightest_mag = lc_real[np.argmin(lc_real["MAG_FPHOT"])]
    for i in np.unique(lc_real["FILTER"]):
        lc_real_filter = lc_real[np.where(lc_real["FILTER"] == i)]
        before_max = len(lc_real_filter[(lc_real_filter["MJD_OBS"] < brightest_mag["MJD_OBS"])]) > 4
        after_max = len(lc_real_filter[(lc_real_filter["MJD_OBS"] > brightest_mag["MJD_OBS"])]) > 6

        plt.errorbar(lc_real_filter["MJD_OBS"], lc_real_filter["MAG_FPHOT"], lc_real_filter["MAGERR_FPHOT"], fmt = "o", color = filter_dict[i], label = i)
        #plt.scatter(lc_real_filter["MJD_OBS"], np.power(10, lc_real_filter["MAG_FPHOT"] / (-2.5)), color = filter_dict[i], label = i)
    plt.legend()
    plt.gca().invert_yaxis()
    plt.title("{} z = {:.3f} SPECTYPE = {}".format(row["STANDARD_NAME"], row["Z"], row["SPECTYPE"]))
    p.savefig(curfig)
    plt.close()
p.close() 